In [1]:
#Dependencies
import pandas as pd
import numpy as np
import json
import requests
import time
from pandas import json_normalize
import os
import csv

In [2]:
dec_data = pd.read_csv("dec_sample_data.csv")
airports = pd.read_csv("airports.csv")

In [3]:
airports.head()

,Airport Name,City name,Country name,Airport Code,lat,lng
0,Goroka Airport,Goroka,Papua New Guinea,GKA,-6.081690,145.391998
1,Madang Airport,Madang,Papua New Guinea,MAG,-5.207080,145.789001
2,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,-5.826790,144.296005
3,Nadzab Airport,Nadzab,Papua New Guinea,LAE,-6.569803,146.725977
4,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,-9.443380,147.220001


In [4]:
dec_data.head()

,departure,arrival,date,weekday,dep_time,gate_dep_time,arr_time,gate_arr_time,sched_time,act_tot_time,dep_delay,arr_delay,timediff
0,CVG,BOS,20201201,2,730,851,939,1133,129,162,81,114,33
1,CVG,BOS,20201203,4,730,729,939,953,129,144,-1,14,15
2,CVG,BOS,20201204,5,730,726,939,914,129,108,-4,-25,-21
3,CVG,BOS,20201206,7,730,723,939,932,129,129,-7,-7,0
4,CVG,BOS,20201207,1,730,720,939,919,129,119,-10,-20,-10


In [5]:
dec_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397850 entries, 0 to 397849
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   departure      397850 non-null  object
 1   arrival        397850 non-null  object
 2   date           397850 non-null  int64 
 3   weekday        397850 non-null  int64 
 4   dep_time       397850 non-null  int64 
 5   gate_dep_time  397850 non-null  int64 
 6   arr_time       397850 non-null  int64 
 7   gate_arr_time  397850 non-null  int64 
 8   sched_time     397850 non-null  int64 
 9   act_tot_time   397850 non-null  int64 
 10  dep_delay      397850 non-null  int64 
 11  arr_delay      397850 non-null  int64 
 12  timediff       397850 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 39.5+ MB


In [6]:
dec_data["ident"] = dec_data["departure"]

In [7]:
airport_data = pd.read_csv("airport-codes.csv")

In [8]:
airport_data.head()

,City name,Airport Code,Airport Name,Country name,Country Abbrev.,World Area Code,ident,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,108 Mile Ranch,ZMH,108 Mile Ranch,Canada,CA,906,ZMH,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aachen,AAH,Aachen/Merzbruck,Germany,DE,429,AAH,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
clean_airport_data = airport_data.dropna(how="all")



In [10]:
clean_airport_data.drop(labels="Unnamed: 7", axis=1)

,City name,Airport Code,Airport Name,Country name,Country Abbrev.,World Area Code,ident
1,108 Mile Ranch,ZMH,108 Mile Ranch,Canada,CA,906,ZMH
3,Aachen,AAH,Aachen/Merzbruck,Germany,DE,429,AAH
5,Aachen,ZIU,Railway,Germany,DE,429,ZIU
7,Aalborg,AAL,Aalborg,Denmark,DK,419,AAL
9,Aalesund,AES,Vigra,Norway,NO,465,AES
...,...,...,...,...,...,...,...
20213,New Orleans,MSY,Louis Armstrong New Orleans International Airport,United States,US,67,MSY
20215,Dammam,DMM,King Fahd International Airport,Saudi Arabia,SA,670,DMM
20217,Pittsburgh,PIT,Pittsburgh International Airport,United States,US,67,PIT
20219,Kinshasa,FIH,N’Djili Airport,D. R. .Congo (Zaire),CD,517,FIH


In [11]:
dec_data_new = dec_data.merge(clean_airport_data, how="left", on="ident")

In [12]:
dec_data_new.head()

,departure,arrival,date,weekday,dep_time,gate_dep_time,arr_time,gate_arr_time,sched_time,act_tot_time,...,arr_delay,timediff,ident,City name,Airport Code,Airport Name,Country name,Country Abbrev.,World Area Code,Unnamed: 7
0,CVG,BOS,20201201,2,730,851,939,1133,129,162,...,114,33,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67,NaN
1,CVG,BOS,20201203,4,730,729,939,953,129,144,...,14,15,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67,NaN
2,CVG,BOS,20201204,5,730,726,939,914,129,108,...,-25,-21,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67,NaN
3,CVG,BOS,20201206,7,730,723,939,932,129,129,...,-7,0,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67,NaN
4,CVG,BOS,20201207,1,730,720,939,919,129,119,...,-20,-10,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67,NaN


In [13]:
dec_data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397850 entries, 0 to 397849
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   departure        397850 non-null  object 
 1   arrival          397850 non-null  object 
 2   date             397850 non-null  int64  
 3   weekday          397850 non-null  int64  
 4   dep_time         397850 non-null  int64  
 5   gate_dep_time    397850 non-null  int64  
 6   arr_time         397850 non-null  int64  
 7   gate_arr_time    397850 non-null  int64  
 8   sched_time       397850 non-null  int64  
 9   act_tot_time     397850 non-null  int64  
 10  dep_delay        397850 non-null  int64  
 11  arr_delay        397850 non-null  int64  
 12  timediff         397850 non-null  int64  
 13  ident            397850 non-null  object 
 14  City name        392628 non-null  object 
 15  Airport Code     392628 non-null  object 
 16  Airport Name     392628 non-null  obje

In [14]:
final_dec_data = dec_data_new.drop(labels="Unnamed: 7", axis=1)


In [15]:
final_dec_data.head()

,departure,arrival,date,weekday,dep_time,gate_dep_time,arr_time,gate_arr_time,sched_time,act_tot_time,dep_delay,arr_delay,timediff,ident,City name,Airport Code,Airport Name,Country name,Country Abbrev.,World Area Code
0,CVG,BOS,20201201,2,730,851,939,1133,129,162,81,114,33,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67
1,CVG,BOS,20201203,4,730,729,939,953,129,144,-1,14,15,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67
2,CVG,BOS,20201204,5,730,726,939,914,129,108,-4,-25,-21,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67
3,CVG,BOS,20201206,7,730,723,939,932,129,129,-7,-7,0,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67
4,CVG,BOS,20201207,1,730,720,939,919,129,119,-10,-20,-10,CVG,Cincinnati,CVG,Cinci./Nrthrn Kentucky,United States,US,67


In [16]:
dec_data_final = final_dec_data.merge(airports, how="left", on="Airport Code")

In [17]:
dec_data_final.drop('Airport Name_x', axis='columns', inplace=True)
dec_data_final.drop('Country name_x', axis='columns', inplace=True)
dec_data_final.drop('City name_y', axis='columns', inplace=True)

In [18]:
dec_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397850 entries, 0 to 397849
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   departure        397850 non-null  object 
 1   arrival          397850 non-null  object 
 2   date             397850 non-null  int64  
 3   weekday          397850 non-null  int64  
 4   dep_time         397850 non-null  int64  
 5   gate_dep_time    397850 non-null  int64  
 6   arr_time         397850 non-null  int64  
 7   gate_arr_time    397850 non-null  int64  
 8   sched_time       397850 non-null  int64  
 9   act_tot_time     397850 non-null  int64  
 10  dep_delay        397850 non-null  int64  
 11  arr_delay        397850 non-null  int64  
 12  timediff         397850 non-null  int64  
 13  ident            397850 non-null  object 
 14  City name_x      392628 non-null  object 
 15  Airport Code     392628 non-null  object 
 16  Country Abbrev.  392628 non-null  obje

In [19]:
dec_data_final.to_csv('final_dec.csv')

In [20]:
means = dec_data_final.groupby('departure').mean()

In [21]:
means.head()

,date,weekday,dep_time,gate_dep_time,arr_time,gate_arr_time,sched_time,act_tot_time,dep_delay,arr_delay,timediff,lat,lng
departure,,,,,,,,,,,,,
ABE,2.020122e+07,3.937677,1131.617564,1121.713881,1328.526912,1301.611898,131.073654,117.291785,17.532578,7.065156,-10.467422,40.652100,-75.440804
ABI,2.020122e+07,3.948454,1078.845361,1067.618557,1200.690722,1183.283505,72.567010,69.067010,4.994845,2.793814,-2.201031,32.411301,-99.681900
ABQ,2.020122e+07,3.933590,1207.605390,1163.368624,1407.117421,1349.045236,115.816169,108.544755,1.248316,-2.444658,-3.692974,35.040199,-106.609001
ABR,2.020122e+07,3.919355,940.161290,946.048387,1074.129032,1047.741935,74.612903,62.983871,19.193548,9.967742,-9.225806,45.449100,-98.421799
ABY,2.020122e+07,3.888889,1203.000000,1192.522222,1304.933333,1288.144444,61.933333,54.733333,0.188889,-7.011111,-7.200000,31.535500,-84.194504


In [22]:
mean_merge = means.filter(['departure', "arr_delay"], axis=1)

In [23]:
mean_merge.rename(columns={"arr_delay": "mean_arr_delay"}, inplace = True)
mean_merge.head()

,mean_arr_delay
departure,
ABE,7.065156
ABI,2.793814
ABQ,-2.444658
ABR,9.967742
ABY,-7.011111


In [28]:
final = mean_merge.merge(dec_data_final, how="left", on="departure")
final2 = final.drop_duplicates(subset="departure")

In [29]:
final2.head()

,departure,mean_arr_delay,arrival,date,weekday,dep_time,gate_dep_time,arr_time,gate_arr_time,sched_time,...,timediff,ident,City name_x,Airport Code,Country Abbrev.,World Area Code,Airport Name_y,Country name_y,lat,lng
0,ABE,7.065156,ATL,20201201,2,615,612,835,818,140,...,-14,ABE,Allentown,ABE,US,67,Lehigh Valley International Airport,United States,40.652100,-75.440804
353,ABI,2.793814,DFW,20201201,2,808,802,911,903,63,...,-2,ABI,Abilene,ABI,US,67,Abilene Regional Airport,United States,32.411301,-99.681900
547,ABQ,-2.444658,DFW,20201201,2,638,632,935,939,117,...,10,ABQ,Albuquerque,ABQ,US,67,Albuquerque International Sunport,United States,35.040199,-106.609001
1586,ABR,9.967742,MSP,20201229,2,610,609,727,712,77,...,-14,ABR,Aberdeen,ABR,US,67,Aberdeen Regional Airport,United States,45.449100,-98.421799
1648,ABY,-7.011111,ATL,20201201,2,1300,1255,1402,1352,62,...,-5,ABY,Albany,ABY,US,67,Southwest Georgia Regional Airport,United States,31.535500,-84.194504


In [26]:
def df_to_geojson(df, properties, lat='lat', lon='lng'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [31]:
cols = ['departure', "City name_x", "Country name_y", "mean_arr_delay"]
geojson = df_to_geojson(final2, cols)


    

In [34]:
output_filename = 'airport.json'
with open(output_filename, 'w') as json_file:
    json.dump(geojson, json_file)